In [ ]:
#import of the required libraries
import pandas as pd
import time

from binance import BinanceSocketManager, Client

#the api-key and api-secret are saved in an other document for privacy
%run ./Binance_Keys.ipynb

#initiation of the connection to the binance api
client = Client(api_key,api_secret)
bsm = BinanceSocketManager(client)

In [ ]:
#input of the variables that should be used for the trading strategy
investment_amt = float(input('How much do you want to invest in USDT: '))
SL = float(input('When should the loss be realised? i.e. enter 0.98 if after a loss of 2%. \nStop Loss at: '))
TS = float(input('When should the trailing stop be activated? \ni.e. enter 0.9975 if after a pullback of \
0.25% from the highest close since the opening of the trade. \nTrailing stop at: '))
Target = float(input('What is the minimum amount of gain required before taking profit? \ni.e. enter 1.01 \
if after a gain of 1% \nTake profit at: '))
Instant_Target = float(input('At which gain percentage should the profit be realised immediately? \ni.e. enter 1.025 \
if after a gain of 2.5% \nTake immediate profit at: '))

In [ ]:
#getting top symbol from the last 24 hours
def get_top_symbol():
    all_pairs = pd.DataFrame(client.get_ticker())
    #sorting by the 24h percentage changes
    all_pairs['priceChangePercent'] = all_pairs['priceChangePercent'].astype(float)
    #filtering out all non 'USDT' pairs
    relev = all_pairs[all_pairs.symbol.str.contains('USDT')].sort_values(by='priceChangePercent', ascending=False)
    #filtering out all leveraged pairs
    non_lev_down = relev[~relev.symbol.str.contains('DOWN')]
    non_lev_down_up = non_lev_down[~non_lev_down.symbol.str.contains('UP')]
    #getting ticker of the current top pair
    top_symbol = non_lev_down_up.iloc[0]['symbol']
    return top_symbol

In [ ]:
#getting 1-min data for the top symbol
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC'))
    #keeping only the listed columns and namig them and setting the time as the index
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [ ]:
#definition of strategy to buy the #1-coin with the most movement and a stop-loss and a take-profit target
def strategy(buy_amt, SL, TS, Target, Instant_Target, open_position=False):
    try:
        #in order to prevent being blocked from the api because of too many request the 
        #requests have to be made in intervals
        time.sleep(2)
        #setting the traded pair as the current top pair from the last 24h
        #asset = get_top_symbol()
        asset = 'BICOUSDT'
        #getting the performance over the last 20min from the top pair
        df = getminutedata(asset, '1m', '20')
        #checking whether sufficient funds are available for trading
        available_usd = [i for i in client.get_account()['balances'] if i['asset'] == 'USDT'][0]['free']
    except:
        #in order to prevent the programm from shutting down a break of 61 seconds has to be made
        #if too many requests have been sent to the binance api
        time.sleep(61)
        #asset = get_top_symbol()
        asset = 'BICOUSDT'
        df = getminutedata(asset, '1m', '20')
        available_usd = [i for i in client.get_account()['balances'] if i['asset'] == 'USDT'][0]['free']
    
    #buy if the performance over the last 20min, 10min and 5 min has been positive
    if (df.Close.pct_change() +1).cumprod().iloc[-1] > 1 and \
    (df.Close.tail(10).pct_change() +1).cumprod().iloc[-1] > 1 and \
    (df.Close.tail(5).pct_change() +1).cumprod().iloc[-1] > 1 and float(available_usd) > investment_amt:
        #setting the buy amount in the top pair
        info = client.get_symbol_info(symbol=asset)
        Lotsize = float([i for i in info['filters'] if i['filterType'] == 'LOT_SIZE'][0]['minQty'])
        prize = float(client.get_symbol_ticker(symbol=asset)['price'])
        buy_quantity = round(round(50/prize/Lotsize) * Lotsize, len(str(Lotsize))-2)
        #placing the order to buy
        order = client.create_order(symbol=asset, side='BUY', type='MARKET', quantity=buy_quantity)
        print(order)
        #saving the buyprice for later calculations
        buyprice = float(order['fills'][0]['price'])
        print(buyprice)
        #setting position to true in order to prevent multiple trades at once
        open_position = True
        
        #while trade is active
        while open_position:
            try:
                #to check for the close conditions of the trade the data from the last 5min has to be available
                df = getminutedata(asset, '1m', '5')
            except:
                print('Script paused, it will continue in 1 min.')
                time.sleep(61)
                df = getminutedata(asset, '1m', '5')
            #creating a benchmark for the trailing stop which stores the highest reached closing value 
            #during the trade
            df['Benchmark'] = df.Close.cummax()
            #setting the trailing stop 
            df['TSL'] = df.Benchmark * TS
            #printing information about the current trade
            print('Current Close is at:', round(df.Close[-1], 4))
            print('First Target Level is at:', round(buyprice * Target, 4))
            print('Trailing Stop Level is at:', round(df.TSL[-1], 4))
            print('Instant Target Level is at:', round(buyprice * Instant_Target, 4))
            print('Current Stop Loss Level is at:', round(buyprice * SL, 4), '\n')
            #in order to prevent being blocked from the api because of too many request the 
            #requests have to be made in intervals
            time.sleep(0.5)
            
            #checking for the close conditions of the trade which are
            #close if price below stop loss
            #close if price above target level and below the trailing stop
            #close if price above the instant target level
            if df.Close[-1] <= buyprice * SL or \
            (df.Close[-1] >= buyprice * Target and df[df.Close < df.TSL].first_valid_index()) or \
            df.Close[-1] >= buyprice * Instant_Target:
                order = client.create_order(symbol=asset, side='SELL', type='MARKET', quantity=buy_quantity)
                print(order)
                sellprice = float(order['fills'][0]['price'])
                profit = (sellprice/buyprice)
                #print performance information about the past trade 
                print('Profit for last trade was:', round((profit - 1) * 100, 2), '%')
                print('Profit for last trade was:', round((sellprice * buy_quantity) - (buyprice * buy_quantity), 2), 'USDT')
                #stop the program for 45 seconds if past trade resulted in a loss
                #in order to prevent over-trading
                if profit < SL:
                    time.sleep(45)
                    break
                #continue immediately if trade was closed due to reaching the instant target level
                if profit > Instant_Target:
                    break
                #if neither of the two conditions have been met a break of 15 seconds is started 
                #before the continuation of the trading in order to prevent over-trading
                else:
                    time.sleep(15)
                    break

In [ ]:
#execute this part if you are content with your inputs and want to start the automated trading
while True:
    strategy(investment_amt, SL, TS, Target, Instant_Target)